In [ ]:
api_key='RGAPI-5fc4f673-301b-4011-9fd2-e375b785f9b7'
import requests

# Retrieving high and low rank leaderboard, puuids and match ids

In [ ]:
import pandas as pd

#get chal, gm, master roots
root= 'https://euw1.api.riotgames.com/'
challenger_root = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
grandmaster_root = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
master_root = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

challenger_response = requests.get(root+challenger_root+'?api_key='+api_key)
grandmaster_response = requests.get(root+grandmaster_root+'?api_key='+api_key)
master_response = requests.get(root+master_root+'?api_key='+api_key)

challenger_df = pd.DataFrame(challenger_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
grandmaster_df=pd.DataFrame(grandmaster_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
master_df=pd.DataFrame(master_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)


#concatenating the dataframes
high_rank_ladder=pd.concat([challenger_df, grandmaster_df, master_df])[0:1000].reset_index(drop=True)
high_rank_ladder=high_rank_ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
high_rank_ladder['rank']+=1

high_rank_ladder

In [ ]:
#get dia, em, plat, gold, silver and bronze

root= 'https://euw1.api.riotgames.com/'
diamond_one_root = 'lol/league/v4/entries/RANKED_SOLO_5x5/DIAMOND/I?page=1&'
emerald_one_root = 'lol/league/v4/entries/RANKED_SOLO_5x5/EMERALD/I?page=1&'
platinum_one_root = 'lol/league/v4/entries/RANKED_SOLO_5x5/PLATINUM/I?page=1&'
gold_one_root = 'lol/league/v4/entries/RANKED_SOLO_5x5/GOLD/I?page=1&'
silver_one_root = 'lol/league/v4/entries/RANKED_SOLO_5x5/SILVER/I?page=1&'
bronze_one_root= 'lol/league/v4/entries/RANKED_SOLO_5x5/BRONZE/I?page=1&'


diamond_one_response = requests.get(root+diamond_one_root+'api_key='+api_key)
emerald_one_response = requests.get(root+emerald_one_root+'api_key='+api_key)
platinum_one_response = requests.get(root+platinum_one_root+'api_key='+api_key)
gold_one_response = requests.get(root+gold_one_root+'api_key='+api_key)
silver_one_response = requests.get(root+silver_one_root+'api_key='+api_key)
bronze_one_response = requests.get(root+bronze_one_root+'api_key='+api_key)

diamond_df = pd.DataFrame(diamond_one_response.json())
emerald_df = pd.DataFrame(emerald_one_response.json())
platinum_df = pd.DataFrame(platinum_one_response.json())
gold_df = pd.DataFrame(gold_one_response.json())
silver_df = pd.DataFrame(silver_one_response.json())
bronze_df = pd.DataFrame(bronze_one_response.json())


diamond_df = diamond_df.sort_values('leaguePoints', ascending=False).reset_index(drop=True)
emerald_df=emerald_df.sort_values('leaguePoints', ascending=False).reset_index(drop=True)
platinum_df = platinum_df.sort_values('leaguePoints', ascending=False).reset_index(drop=True)
gold_df = gold_df.sort_values('leaguePoints', ascending=False).reset_index(drop=True)
silver_df = silver_df.sort_values('leaguePoints', ascending=False).reset_index(drop=True)
bronze_df = bronze_df.sort_values('leaguePoints', ascending=False).reset_index(drop=True)

low_rank_ladder = pd.concat([diamond_df,emerald_df, platinum_df, gold_df, silver_df, bronze_df ]).reset_index(drop=True)

In [ ]:
import time

def get_puuid_from_summoner_id(summonerId=None):
    '''takes summoner id as in parameter and retrieves puuid from endpoint'''
    link=f'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}?api_key={api_key}'

    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        puuid=response.json()['puuid']
        return puuid

In [ ]:
# get every puuid for players in high rank ladder
puuid_list = []

for summoner in high_rank_ladder['summonerId']:
    puuids = get_puuid_from_summoner_id(summonerId=summoner)
    puuid_list.append(puuids)

In [ ]:
#get every puuid for players in low rank ladder
low_puuid_list = []

for summoner in low_rank_ladder['summonerId']:
    puuids = get_puuid_from_summoner_id(summonerId=summoner)
    low_puuid_list.append(puuids)

In [ ]:
def get_match_id_from_puuid(puuid=None):
    """
     Takes a player's puuid as parameter and returns the match id's of their most recent 15 ranked matches
    Queue and type must remain as 420 and ranked respectively as this will only get the ranked matches which is what we want
    """
    link=f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&type=ranked&start=0&count=15&api_key={api_key}'
    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        match_id_list=response.json()
        break
    return match_id_list


In [ ]:
#gets match ids for every
all_match_id_list=[]

for player_puuid in puuid_list:
    match_ids=get_match_id_from_puuid(puuid=player_puuid)
    all_match_id_list.extend(match_ids)

In [ ]:
def get_low_match_id_from_puuid(puuid=None, count=5):
    """
    takes a player's puuid as parameter and outputs the match id's of their most recent 5 ranked matches.
    Queue and type must remain as 420 and ranked respectively as this will only get the ranked matches which is what we want.
    """
    link=f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&type=ranked&start=0&count={count}&api_key={api_key}'
    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        match_id_list=response.json()
        break
    return match_id_list


# Removing duplicates from match lists (they are the same matches)

In [ ]:
from collections import Counter

#count all duplicates and convert to set then back to list to remove them
match_id_counts = Counter(all_match_id_list)

unique_match_id_list = list(set(all_match_id_list))

print(f"Original number of match IDs: {len(all_match_id_list)}")

print(f"Number of unique match IDs after removal: {len(unique_match_id_list)}")


# Retrieivng entire match JSON given a match ID

In [ ]:
def get_match_info_from_id(matchId=None):
    '''takes match id as parameter and returns match json file'''
    link= f'https://europe.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}'
    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        elif response.status_code==200:
            break
        elif response.status_code==404:
            return 0
        else:
            print('Response is bad: '+str(response.status_code))
    match_data=response.json()
    return match_data

# Retrieving specific match information that I am interested in

In [ ]:
import pandas as pd
#https://www.youtube.com/watch?v=CAszRp8yfW4&list=PLnnM9dgSdUvIe0MRJcjbzBaJWFHNJaOh1&index=6
def get_specific_match_info(data):
    '''takes match json file as parameter (data) and processes it to retrieve only the information we want '''
    info = data['info']
    metadata = data['metadata']
    participants = metadata['participants']  # List of player puuids
    teams = info['teams']
    
    #specific stats that applies to every player in the match
    game_duration = info['gameDuration']
    game_ended_in_surrender = info['participants'][0]['gameEndedInSurrender']
    game_ended_in_early_surrender=info['participants'][0]['gameEndedInEarlySurrender']
    blue_side_won = info['participants'][0]['win']
    

    #objectives will be considered separately because its in teamsDTO and not participantDTO

    #blue side objectives
    blue_side_tower = teams[0]['objectives']['tower']['kills']
    blue_side_dragon = teams[0]['objectives']['dragon']['kills']
    blue_side_baron = teams[0]['objectives']['baron']['kills']
    blue_side_riftHerald = teams[0]['objectives']['riftHerald']['kills']
    blue_side_horde = teams[0]['objectives']['horde']['kills']
    blue_side_inhibitor = teams[0]['objectives']['inhibitor']['kills']

    #red side objectives
    red_side_tower = teams[1]['objectives']['tower']['kills']
    red_side_dragon = teams[1]['objectives']['dragon']['kills']
    red_side_baron =teams[1]['objectives']['baron']['kills']
    red_side_riftHerald =teams[1]['objectives']['riftHerald']['kills']
    red_side_horde = teams[1]['objectives']['horde']['kills']
    red_side_inhibitor = teams[1]['objectives']['inhibitor']['kills']


    #build dictionaries for both blue adn red sides stats
    blue_side_stats = {
        'minions_killed': 0,
        'kills': 0,
        'deaths': 0,
        'assists': 0,
        'gold_earned': 0,
        'gold_spent': 0,
        'wards_killed':0,
        'wards_placed':0,
        'vision_score':0
    }

    red_side_stats = {
        'minions_killed': 0,
        'kills': 0,
        'deaths': 0,
        'assists': 0,
        'gold_earned': 0,
        'gold_spent': 0,
        'wards_killed':0,
        'wards_placed':0,
        'vision_score':0
    }
    
    
    #iterate through each players data. if the index of their participant id/puuid is less than 5 then they are on the blue team, otherwise they are on red team
    #side_stats is therefore assigned to each side depending on whether the participant/player id (i) is less than or greater than 5.
    for i, player in enumerate(info['participants']):
        if i < 5:
            side_stats=blue_side_stats
        else:
            side_stats=red_side_stats

        neautral_minions_killed = player['neutralMinionsKilled']
        minions_killed = player['totalMinionsKilled']
        kills = player['kills']
        assists = player['assists']
        deaths = player['deaths']
        gold_earned = player['goldEarned']
        gold_spent = player['goldSpent']
        wards_killed = player['wardsKilled']
        wards_placed=player['wardsPlaced']
        vision_score=player['visionScore']

        #add up the stats for both blue and red sides
        side_stats['minions_killed'] += neautral_minions_killed+minions_killed
        side_stats['assists'] += assists
        side_stats['deaths'] += deaths
        side_stats['kills'] += kills
        side_stats['gold_earned']+=gold_earned
        side_stats['gold_spent']+=gold_spent
        side_stats['wards_killed']+=wards_killed
        side_stats['wards_placed']+=wards_placed
        side_stats['vision_score']+=vision_score

    #create a dataframe with blue and red side stats
    df = pd.DataFrame({
        #game info
        'game_duration':[game_duration],
        'game_ended_in_surrender': [game_ended_in_surrender],
        'game_ended_in_early_surrender': [game_ended_in_early_surrender],
        'blue_side_won': [blue_side_won],

        #blue side
        'total_blue_side_minions_killed': [blue_side_stats['minions_killed']],
        'total_blue_side_kills': [blue_side_stats['kills']],
        'total_blue_side_deaths': [blue_side_stats['deaths']],
        'total_blue_side_assists': [blue_side_stats['assists']],
        'total_blue_side_gold_earned':[blue_side_stats['gold_earned']],
        'total_blue_side_gold_spent':[blue_side_stats['gold_spent']],
        'total_blue_side_wards_killed':[blue_side_stats['wards_killed']],
        'total_blue_side_wards_placed':[blue_side_stats['wards_placed']],
        'total_blue_side_vision_score':[blue_side_stats['vision_score']],
        'total_towers_destroyed_by_blue': [blue_side_tower],
        'total_inhibitors_destroyed_by_blue': [blue_side_inhibitor],
        'total_blue_side_dragons_killed': [blue_side_dragon],
        'total_blue_side_horde': [blue_side_horde],
        'total_blue_side_baron':[blue_side_baron],
        'total_blue_side_riftHerald':[blue_side_riftHerald],

        #red side
        'total_red_side_minions_killed': [red_side_stats['minions_killed']],
        'total_red_side_kills': [red_side_stats['kills']],
        'total_red_side_deaths': [red_side_stats['deaths']],
        'total_red_side_assists': [red_side_stats['assists']],
        'total_red_side_gold_earned':[red_side_stats['gold_earned']],
        'total_red_side_gold_spent':[red_side_stats['gold_spent']],
        'total_red_side_ward_takedowns':[red_side_stats['wards_killed']],
        'total_red_side_wards_placed':[red_side_stats['wards_placed']],
        'total_red_side_vision_score':[red_side_stats['vision_score']],
        'total_towers_destroyed_by_red': [red_side_tower],
        'total_inhibitors_destroyed_by_red': [red_side_inhibitor],
        'total_red_side_dragons_killed': [red_side_dragon],
        'total_red_side_horde': [red_side_horde],
        'total_red_side_baron':[red_side_baron],
        'total_red_side_riftHerald':[red_side_riftHerald]

    })

    return df



In [ ]:
match_ids = unique_low_match_id_list
dfs = pd.DataFrame()  #empty dataframe to hold all our data

#iterate through all match ids and process the matches
for match_id in match_ids:
    try:
        game = get_match_info_from_id(matchId=match_id)     
        if game is None:
            print(f'{match_id} is none. skip')
            continue
        try:
            matchdf=get_specific_match_info(game)
        except KeyError as e:
            if 'horde' in str(e):
                print(f'KeyError with {match_id}: {e}')
                continue
            else:
                raise e
        if matchdf.empty:
            print(f'{match_id} is empty. skip')
            continue

        dfs= pd.concat([dfs,matchdf])

    except (TypeError, KeyError, IndexError) as e:
        print(f'{match_id} raised an error. skip')

In [ ]:
dfs

In [ ]:
dfs.to_csv('low_rank_final.csv')